# 실습 10: SageMaker Studio와 함께 SageMaker 파이프라인 및 SageMaker Model Registry 사용

이 실습에서는 Amazon SageMaker 파이프라인을 생성하고 해당 파이프라인의 진행률을 모니터링합니다. 그리고 기계 학습(ML) 프로세스가 사용하거나 생성하는 몇 가지 아티팩트도 찾아서 살펴봅니다.

시간이 되면 파이프라인에서 생성된 모델의 계보 세부 정보도 검토할 수 있습니다.

## 과제 2.1: 환경 설정

SageMaker 파이프라인을 생성하기 전에 먼저 필요한 패키지를 설치하고 모듈을 가져온 후 지원 파일을 스테이징하여 환경을 준비해야 합니다. 이 파이프라인은 특성 그룹을 사용하도록 설계되었으므로 Amazon SageMaker Feature Store에서 특성 그룹도 생성하며 Data Wrangler 흐름을 실행하여 환경을 준비합니다. 

이 과제의 셀을 실행하여 다음 작업을 수행합니다.
- 종속성 설치
- 필요한 모듈 가져오기
- Amazon Simple Storage Service(Amazon S3)에 데이터 및 코드 복사
- 특성 그룹 생성
- 특성 그룹에 특성 수집

### 과제 2.1.1: 종속성 설치

In [ ]:
#install dependencies
%pip install --upgrade pip 
%pip install pytest-astropy ==  0.7.0
%pip install rsa == 4.7.2
%pip install PyYAML
!apt update && apt install -y git
%pip install git+https://github.com/aws-samples/ml-lineage-helper

### 과제 2.1.2: 모듈 가져오기

In [ ]:
#import-modules
import os
import json
import boto3
import sagemaker
import sagemaker_datawrangler
import sagemaker.session
import datetime as dt
import pandas as pd
import time
from time import gmtime, strftime
import uuid
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.transformer import Transformer
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from ml_lineage_helper import *
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor
from sagemaker.network import NetworkConfig
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

In [ ]:
#create sessions
boto_session  =  boto3.Session()
sagemaker_session = sagemaker.Session()


In [ ]:
#create clients
s3_client = boto3.client('s3')
featurestore_runtime = boto3.client('sagemaker-featurestore-runtime')
sagemaker_client = boto3.client('sagemaker')

In [ ]:
#feature store session
feature_store_session = Session(
    boto_session = boto_session,
    sagemaker_client = sagemaker_client,
    sagemaker_featurestore_runtime_client = featurestore_runtime
)

In [ ]:
#set global variables
default_bucket = sagemaker_session.default_bucket()
region = boto_session.region_name
role = sagemaker.get_execution_role()

### 과제 2.1.3 Amazon S3에 실습 파일 복사 

In [ ]:
# Upload files to default bucket
s3_client.put_object(Bucket = default_bucket, Key = 'data/')
s3_client.put_object(Bucket = default_bucket, Key = 'input/code/')
s3_client.upload_file('pipelines/data/storedata_total.csv', default_bucket, 'data/storedata_total.csv')
s3_client.upload_file('pipelines/input/code/evaluate.py', default_bucket, 'input/code/evaluate.py')
s3_client.upload_file('pipelines/input/code/generate_config.py', default_bucket, 'input/code/generate_config.py')
s3_client.upload_file('pipelines/input/code/processfeaturestore.py', default_bucket, 'input/code/processfeaturestore.py')

# Preview the dataset
print('Dataset preview:')
customer_data = pd.read_csv('pipelines/data/storedata_total.csv')
customer_data.head()

### 과제 2.1.4: 특성 그룹 생성

이 작업에서는 데이터에 대한 특성 그룹을 생성합니다. 먼저 데이터의 스키마를 만듭니다. 이 실습의 경우, 스키마는 **name** 열과 변수의 **type** 열에서 생성되어야 합니다.

In [ ]:
#set-up-feature-store-variables
record_identifier_feature_name = 'FS_ID'
event_time_feature_name = 'FS_time'

column_schemas = [
    {
        "name": "retained",
        "type": "long"
    },
    {
        "name": "esent",
        "type": "long"
    },
    {
        "name": "eopenrate",
        "type": "float"
    },
    {
        "name": "eclickrate",
        "type": "float"
    },
    {
        "name": "avgorder",
        "type": "float"
    },
    {
        "name": "ordfreq",
        "type": "float"
    },
    {
        "name": "paperless",
        "type": "long"
    },
    {
        "name": "refill",
        "type": "long"
    },
    {
        "name": "doorstep",
        "type": "long"
    },
    {
        "name": "first_last_days_diff",
        "type": "long"
    },
    {
        "name": "created_first_days_diff",
        "type": "long"
    },
    {
        "name": "favday_Friday",
        "type": "long"
    },
    {
        "name": "favday_Monday",
        "type": "long"
    },
    {
        "name": "favday_Saturday",
        "type": "long"
    },
    {
        "name": "favday_Sunday",
        "type": "long"
    },
    {
        "name": "favday_Thursday",
        "type": "long"
    },
    {
        "name": "favday_Tuesday",
        "type": "long"
    },
    {
        "name": "favday_Wednesday",
        "type": "long"
    },
    {
        "name": "city_BLR",
        "type": "long"
    },
    {
        "name": "city_BOM",
        "type": "long"
    },
    {
        "name": "city_DEL",
        "type": "long"
    },
    {
        "name": "city_MAA",
        "type": "long"
    },
    {
        "name": "FS_ID",
        "type": "long"
    },
    {
        "name": "FS_time",
        "type": "float"
    }
]


이제 특성 그룹을 생성합니다.

In [ ]:
# Flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = 'featureengineer'
flow_export_id = f"{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"

# Feature group name, with flow_name and a unique id. You can give it a customized name
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"

# SageMaker Feature Store writes the data in the offline store of a Feature Group to a 
# Amazon S3 location owned by you.
feature_store_offline_s3_uri = 's3://' + default_bucket

# Controls if online store is enabled. Enabling the online store allows quick access to 
# the latest value for a record by using the GetRecord API.
enable_online_store = True

In [ ]:
#create-feature-group
default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name = column_schema['name'], 
        feature_type = column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]


print(f"Feature Group Name: {feature_group_name}")

# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup = 'primary',
        ConfigurationUpdates = {
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

feature_group = FeatureGroup(
    name = feature_group_name, sagemaker_session = feature_store_session, feature_definitions = feature_definitions)

feature_group.create(
    s3_uri = feature_store_offline_s3_uri,
    record_identifier_name = record_identifier_feature_name,
    event_time_feature_name = event_time_feature_name,
    role_arn = role,
    enable_online_store = enable_online_store
)

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for feature group creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"Feature Group {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group = feature_group)


### 과제 2.1.5: 특성 수집

이 프로세스 실행을 완료하려면 약 8분이 소요됩니다.

In [ ]:
#populate-feature-store
column_list = ['retained','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday', 'favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA','FS_ID','FS_time']
lab_test_data = pd.read_csv('featureengineer_data/store_data_processed.csv', names = (column_list), header = 1)
feature_group.ingest(data_frame = lab_test_data, wait = True)

## 과제 2.2: SageMaker 파이프라인 생성 및 실행

환경을 설정했으므로 이제 SageMaker 파이프라인을 구성, 생성 및 시작합니다. 

SageMaker 파이프라인은 종속 단계 집합을 실행하는 워크플로입니다. 단계에서는 입력을 수락하고 출력을 전송할 수 있으므로 단계 간에 데이터와 기타 자산을 전달할 수 있습니다. 

아래 셀을 실행하여 다음 작업을 수행합니다.
- 파이프라인을 구성하기 위해 필요한 변수 정의
- SageMaker 세션 구성
- 파이프라인 단계 정의
- 파이프라인 구성
- 파이프라인 생성
- 파이프라인 시작
- 파이프라인 설명
- 파이프라인 실행이 완료될 때까지 노트북이 처리되지 않도록 대기 이벤트 생성

### 과제 2.2.1: 파이프라인이 사용하는 변수 설정

In [ ]:

#pipeline-variables
feature_group_name = feature_group.name
model_name = "Churn-model"

sklearn_processor_version = "0.23-1"
model_package_group_name = "ChurnModelPackageGroup"
pipeline_name = "ChurnModelSMPipeline"

processing_instance_count = ParameterInteger(
    name = "ProcessingInstanceCount",
    default_value = 1
    )

processing_instance_type = ParameterString(
        name = "ProcessingInstanceType",
        default_value = "ml.m5.xlarge"
    )

training_instance_type = ParameterString(
        name = "TrainingInstanceType",
        default_value = "ml.m5.xlarge"
    )

input_data = ParameterString(
        name = "InputData",
        default_value = "s3://{}/data/storedata_total.csv".format(default_bucket), 
    )

batch_data = ParameterString(
        name = "BatchData",
        default_value = "s3://{}/data/batch/batch.csv".format(default_bucket),
    )

### 과제 2.2.2: 파이프라인 구성

이 과제에서는 고객 유지 또는 이탈 가능성을 평가하는 모델을 생성하기 위해 이름이 **ChurnModelPipeline**인 파이프라인을 정의합니다. 이 파이프라인에는 9개 단계가 포함됩니다. 

파이프라인의 각 단계에서는 특정 작업 유형을 실행합니다. 작업에 필요한 입력은 작업 유형에 따라 다릅니다. SageMaker 파이프라인 단계 유형에 관한 자세한 내용은 [단계 유형](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#build-and-manage-steps-types)을 참조하세요.

다음 셀의 코드를 검토하여 각 단계가 정의된 방식을 파악합니다.

**ChurnModelProcess** 단계는 **step_process** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Processing. ProcessingStep() 클래스를 사용하여 처리 작업을 정의합니다.
- **Processor**: SKLearnProcessor입니다.
- **Destination**: 기본 S3 버킷의 폴더로 출력이 전송됩니다.
- **Job Arguments**: 이 단계에서는 특성 저장소를 사용하여 데이터 집합을 처리합니다.
- **Code**: 기본 S3 버킷에 있는 **processfeaturestore.py**입니다.


In [ ]:
#configure-processing-step
# Run a scikit-learn script to do data processing on SageMaker 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version = sklearn_processor_version,
        instance_type = processing_instance_type.default_value, 
        instance_count = processing_instance_count,
        sagemaker_session = sagemaker_session,
        role = role,
    )

# Inputs, outputs, and code are parameters to the processor
# step_* will become the pipeline steps toward the end of the cell
# in this case, use the feature store as input, so there is no externalinput
step_process = ProcessingStep(
        name = "ChurnModelProcess",
        processor = sklearn_processor,
        outputs = [
            ProcessingOutput(output_name = "train", source = "/opt/ml/processing/train",\
                             destination = f"s3://{default_bucket}/output/train" ),
            ProcessingOutput(output_name = "validation", source = "/opt/ml/processing/validation",\
                            destination = f"s3://{default_bucket}/output/validation"),
            ProcessingOutput(output_name = "test", source = "/opt/ml/processing/test",\
                            destination = f"s3://{default_bucket}/output/test"),
            ProcessingOutput(output_name = "batch", source = "/opt/ml/processing/batch",\
                            destination = f"s3://{default_bucket}/data/batch"),
            ProcessingOutput(output_name = "baseline", source = "/opt/ml/processing/baseline",\
                            destination = f"s3://{default_bucket}/input/baseline")
        ],
        job_arguments = ["--featuregroupname",feature_group_name,"--default-bucket",default_bucket,"--region",region],
        code = f"s3://{default_bucket}/input/code/processfeaturestore.py",
    )

**ChurnHyperParameterTuning** 단계는 **step_tuning** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Tuning. TuningStep() 클래스를 사용하여 튜닝 작업을 정의합니다.
- **Tuner**: 이 작업에서는 XGBoost 프레임워크를 사용합니다.
- **Inputs**: 이 작업은 ChurnModelProcess 단계인 **step_process**에서 생성된 훈련 및 검증 데이터를 사용합니다.

In [ ]:
#configure-churn-hyperparameter-tuning
# Training/tuning step for generating model artifacts
model_path = f"s3://{default_bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework = "xgboost",
    region = region,
    version = "1.5-1",
    py_version = "py3",
    instance_type = training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
    }

xgb_train = Estimator(
    image_uri = image_uri,
    instance_type = training_instance_type,
    instance_count = 1,
    hyperparameters = fixed_hyperparameters,
    output_path = model_path,
    base_job_name = f"churn-train",
    sagemaker_session = sagemaker_session,
    role = role
    )

In [ ]:
#Tuning steps
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
    }
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "ChurnHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs = 2, max_parallel_jobs = 2),
    inputs = {
            "train": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
            "validation": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
        },
    )

**ChurnEvalBestModel** 단계는 **step_eval** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Processing입니다.
- **Processor**: ScriptProcessor입니다.
- **Inputs**: 이 작업은 ChurnHyperParameterTuning(**step_tuning**)의 상위 모델과 ChurnModelProcess(**step_process**)의 테스트 출력을 사용합니다.
- **Outputs**: 출력은 기본 S3 버킷에 작성됩니다.
- **Code**: Amazon S3에 있는 **evaluate.py** 스크립트가 평가에 사용됩니다.

In [ ]:
#configure-churn-best-model
evaluation_report = PropertyFile(
    name = "ChurnEvaluationReport",
    output_name = "evaluation",
    path = "evaluation.json",
)

script_eval = ScriptProcessor(
    image_uri = image_uri,
    command = ["python3"],
    instance_type = processing_instance_type,
    instance_count = 1,
    base_job_name = "script-churn-eval",
    role = role,
    sagemaker_session = sagemaker_session,
)

step_eval = ProcessingStep(
    name = "ChurnEvalBestModel",
    processor = script_eval,
    inputs = [
        ProcessingInput(
            source = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
            destination = "/opt/ml/processing/model"
        ),
        ProcessingInput(
            source = step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination = "/opt/ml/processing/test"
        )
    ],
    outputs = [
        ProcessingOutput(output_name = "evaluation", source = "/opt/ml/processing/evaluation",\
                            destination = f"s3://{default_bucket}/output/evaluation"),
    ],
    code = f"s3://{default_bucket}/input/code/evaluate.py",
    property_files = [evaluation_report],
)

**ChurnCreateModel** 단계는 **step_create_model** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Model. Model() 클래스를 사용하여 모델 작업을 정의합니다.
- **Model**: 이 단계에서 사용되는 모델은 이전에 정의한 **model** 변수에서 정의됩니다. **model** 변수는 ChurnHyperParameterTuning(**step_tuning**)에서 생성된 상위 모델을 사용합니다.
- **Inputs**: 입력에는 인스턴스 유형과 액셀러레이터 유형이 포함됩니다.

In [ ]:
#configure-model-creation
model = Model(
    image_uri = image_uri,        
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0,s3_bucket = default_bucket,prefix = "output"),
    name = model_name,
    sagemaker_session = sagemaker_session,
    role = role,
)

inputs = CreateModelInput(
    instance_type = "ml.m5.large",
    accelerator_type = "ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name = "ChurnCreateModel",
    model = model,
    inputs = inputs,
)

**ChurnModelConfigFile** 단계는 **step_config_file** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Processing입니다.
- **Processor**: ScriptProcessor입니다.
- **Code**: 기본 S3 버킷에 있는 **generate_config.py**입니다.
- **Job Arguments**: 작업 인수에는 **ChurnCreateModel**에서 생성된 모델, 바이어스 보고서 경로, 기본 버킷, 샘플 수, 처리에 사용되는 인스턴스 수가 포함됩니다.
- **Depends On**: 모델 생성이 완료될 때까지는 이 작업을 실행할 수 없습니다.

In [ ]:

#configure-script-processing
bias_report_output_path = f"s3://{default_bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
analysis_config_path = f"s3://{default_bucket}/clarify-output/bias/analysis_config.json"
clarify_image = sagemaker.image_uris.retrieve(framework = 'sklearn', version = sklearn_processor_version, region = region)

#custom_image_uri = None
script_processor = ScriptProcessor(
    command = ['python3'],
    image_uri = clarify_image,
    role = role,
    instance_count = 1,
    instance_type = processing_instance_type,
    sagemaker_session = sagemaker_session,
)

step_config_file = ProcessingStep(
    name = "ChurnModelConfigFile",
    processor = script_processor,
    code = f"s3://{default_bucket}/input/code/generate_config.py",
    job_arguments = ["--modelname", step_create_model.properties.ModelName, "--bias-report-output-path", bias_report_output_path, "--clarify-instance-type", clarify_instance_type,\
                  "--default-bucket", default_bucket, "--num-baseline-samples", "50", "--instance-count", "1"],
    depends_on = [step_create_model.name]
)

**ChurnTransform** 단계는 **step_transform** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Transform. TransformStep() 클래스를 사용하여 변환 작업을 정의합니다.
- **Transformer**: 변환기 세부 정보는 앞에서 정의한 **transformer** 변수에서 설정됩니다. 이 변수는 ChurnCreateModel(**step_create_model**)에서 생성된 모델을 사용합니다.
- **Inputs**: 이전 과제를 수행할 때 노트북에서 정의한 변환되는 데이터(batch.csv)입니다. 입력에는 파일 형식과 파일 분할 방법도 포함됩니다.

In [ ]:
#configure-inference
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type = "ml.m5.xlarge",
    instance_count = 1,
    assemble_with = "Line",
    accept = "text/csv",    
    output_path = f"s3://{default_bucket}/ChurnTransform"
    )

step_transform = TransformStep(
    name = "ChurnTransform",
    transformer = transformer,
    inputs = TransformInput(data = batch_data, content_type = "text/csv", join_source = "Input", split_type = "Line")
    )

**ClarifyProcessingStep** 단계는 **step_clarify** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Processing입니다.
- **Processor**: 이 작업에서는 SageMakerClarifyProcessor를 사용합니다. **clarify_processor** 변수에서 프로세서 구성을 검토할 수 있습니다.
- **Inputs**: 입력은 **data_input** 및 **congif_input** 변수에서 정의됩니다.
- **Outputs**: 출력은 기본 버킷의 폴더에 작성됩니다. 
- **Depends On**: **ChurnModelConfigFile**을 사용하여 Amazon SageMaker Clarify에 필요한 구성 파일을 생성할 때까지는 이 작업을 실행할 수 없습니다. 

In [ ]:
#configure-clarify-processing
data_config = sagemaker.clarify.DataConfig(
s3_data_input_path = f's3://{default_bucket}/output/train/train.csv',
s3_output_path = bias_report_output_path,
    label = 0,
    headers = ['target','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday','favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA'],
    dataset_type = "text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role = role,
    instance_count = 1,
    instance_type = clarify_instance_type,
    sagemaker_session = sagemaker_session,
)

config_input = ProcessingInput(
    input_name = "analysis_config",
    source=analysis_config_path,
    destination = "/opt/ml/processing/input/analysis_config",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_compression_type = "None",
    )

data_input = ProcessingInput(
    input_name = "dataset",
    source = data_config.s3_data_input_path,
    destination = "/opt/ml/processing/input/data",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_data_distribution_type = data_config.s3_data_distribution_type,
    s3_compression_type = data_config.s3_compression_type,
)

result_output = ProcessingOutput( 
    source = "/opt/ml/processing/output",
    destination = data_config.s3_output_path,
    output_name = "analysis_result",
    s3_upload_mode = "EndOfJob",
)

step_clarify = ProcessingStep(
    name = "ClarifyProcessingStep",
    processor = clarify_processor,
    inputs = [data_input, config_input],
    outputs = [result_output],
    depends_on = [step_config_file.name]
)

**RegisterChurnModel** 단계는 **step_register** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: RegisterModel. RegisterModel() 클래스를 사용하여 등록 작업을 정의합니다.
- **Estimator**: 추정기는 셀 앞부분의 **xgbtrain** 변수에서 정의됩니다.
- **Model Data**: **ChurnHyperParameterTuning**에서 반환되는 모델 URI입니다.
- **Content Types**: text/csv입니다.
- **Response Types**: text/csv입니다.
- **Inference Instance**: 추론 처리에 사용할 인스턴스 유형입니다.
- **Transform Instance**: 변환을 처리하는 데 사용할 인스턴스 유형입니다.
- **Model Package Group Name**: 모델 버전 그룹을 저장할 그룹의 이름입니다.
- **Model Metrics**: 모델 지표의 위치를 정의합니다. 지표에 포함된 파일은 SageMaker Clarify 바이어스 보고서, SageMaker Clarify 설명 가능성 보고서 및 모델 평가입니다. 

In [ ]:
#configure-model-registry
model_statistics = MetricsSource(
    s3_uri = "s3://{}/output/evaluation/evaluation.json".format(default_bucket),
    content_type = "application/json"
    )
explainability = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    )

bias = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    ) 

model_metrics = ModelMetrics(
    model_statistics = model_statistics,
    explainability = explainability,
    bias = bias
)

step_register = RegisterModel(
    name = "RegisterChurnModel",
    estimator = xgb_train,
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
    content_types = ["text/csv"],
    response_types = ["text/csv"],
    inference_instances = ["ml.t2.medium", "ml.m5.large"],
    transform_instances = ["ml.m5.large"],
    model_package_group_name = model_package_group_name,
    model_metrics = model_metrics,
)

**CheckAUCScoreChurnEvaluation** 단계는 **step_cond** 변수에서 정의됩니다. 

단계 구성에는 다음 항목이 포함됩니다.
- **Type**: Condition. ConditionStep() 클래스를 사용하여 조건 작업을 정의합니다.
- **Conditions**: **ChurnEvalBestModel**의 출력이 0.75보다 크면 이 조건이 True로 평가됩니다.
- **If Steps**: 조건이 True로 평가되면 실행할 단계의 목록입니다.
- **Else Steps**: 조건이 False로 평가되면 실행할 단계의 목록입니다. 이 목록은 비어 있습니다. 즉, 조건이 충족되지 않으면 파이프라인 처리가 중지됩니다.

In [ ]:
%%capture
cond_lte = ConditionGreaterThan(
    left = JsonGet(
        step = step_eval,
        property_file = evaluation_report,
        json_path = "binary_classification_metrics.auc.value"
    ),
    right = 0.75,
)

step_cond = ConditionStep(
    name = "CheckAUCScoreChurnEvaluation",
    conditions = [cond_lte],
    if_steps = [step_create_model, step_config_file, step_transform, step_clarify, step_register],
    else_steps = [],
)

### 과제 2.2.3: 파이프라인 정의

단계를 정의한 후에는 **pipeline** 변수에서 파이프라인을 구성합니다. 앞에서 정의한 단계가 파이프라인 정의로 전달되는 방식을 잘 살펴보세요.

In [ ]:
 #define pipeline function
def get_pipeline(
    region,
    role = None,
    default_bucket = None,
    model_package_group_name = "ChurnModelPackageGroup",
    pipeline_name = "ChurnModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version = None
    ):

    #configure pipeline instance
    pipeline = Pipeline(
        name = pipeline_name,
        parameters = [
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps = [step_process, step_tuning, step_eval, step_cond],
        sagemaker_session = sagemaker_session
    )
    return pipeline


### 과제 2.2.4: 파이프라인 생성

In [ ]:
 #create pipeline using function
pipeline = get_pipeline(
  region = region,
    role = role,
    default_bucket = default_bucket,
    model_package_group_name = model_package_group_name,
    pipeline_name = pipeline_name,
    custom_image_uri = clarify_image,
    sklearn_processor_version = sklearn_processor_version
)

### 과제 2.2.5: 올바른 IAM 역할을 사용하도록 파이프라인 업데이트 

In [ ]:
#set-iam-role
pipeline.upsert(role_arn = role)

**참고**: 셀을 실행한 후에 다음 경고가 표시되면 무시해도 됩니다.

"No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config"(이 추정기와 관련하여 완료된 훈련 작업이 없습니다. 이 추정기가 워크플로 구성 빌드에만 사용되는지 확인하세요.)

### 과제 2.2.6: 파이프라인 시작

In [ ]:
#start-pipeline
RunPipeline = pipeline.start()

### 과제 2.2.7: 파이프라인 설명

In [ ]:
#describe-pipeline
RunPipeline.describe()

이 파이프라인을 실행하려면 약 35분이 소요됩니다.

파이프라인이 실행되는 동안 다음 과제를 계속 진행하여 Amazon SageMaker Studio 콘솔에서 파이프라인을 살펴봅니다.

## 과제 2.3: 파이프라인 모니터링 및 승인

이 과제에서는 Amazon SageMaker Studio 콘솔을 사용하여 파이프라인을 살펴봅니다.

### 과제 2.3.1: SageMaker Studio에서 파이프라인 모니터링

다음 과제에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_10.ipynb** 탭을 옆쪽으로 끌거나 **lab_10.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 파이프라인 단계를 살펴볼 때 지침을 표시할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 파이프라인 단계 결과 탐색을 완료한 후 **lab_10.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

1. SageMaker Studio에서 **SageMaker Home** 아이콘을 선택합니다.
1. **Pipelines**를 선택합니다.

SageMaker Studio에서 **Pipelines** 탭이 열립니다.

1. **ChurnModelSMPipeline**이라는 파이프라인을 선택합니다. 

SageMaker Studio에서 **ChurnModelSMPipeline** 탭이 열립니다.

1. **ChurnModelSMPipeline** 탭의 **Executions**에서 파이프라인 상태를 마우스 오른쪽 버튼으로 클릭하여 열고 **Open execution details**를 선택합니다. 

SageMaker Studio에서 **Directed Acyclic Graph**(DAG) 페이지가 열립니다.

DAG(Directed Acyclic Graph)에 파이프라인 워크플로와 진행률이 표시됩니다. DAG에서는 각 단계 상태가 고유한 색상으로 표시됩니다. 단계 색상 표시기는 다음과 같습니다.
- **회색**: 실행 대기 중.
- **파란색**: 실행 중.
- **녹색**: 성공적으로 완료됨.
- **빨간색**: 오류.

여기서 살펴보는 각 단계의 단계 세부 정보 창에는 동일한 탭 4개가 있습니다. 탭 제목은 동일하지만 각 탭의 내용은 작업 유형과 작업 구성에 따라 달라집니다.

- **Input**: 작업으로 전달된 입력이 포함되어 있는 탭입니다. 입력의 예로는 인스턴스 유형, AWS Identity and Access Management(IAM) 역할 또는 작업 실행에 필요한 인수를 지정하는 파라미터가 포함됩니다. 코드, 데이터 집합, Docker 이미지 등의 파일도 입력의 예에 포함됩니다.
- **Output**: 작업에서 생성된 출력이 표시되는 탭입니다. 출력의 예로는 지표, 차트, 파일, 평가 결과 등이 있습니다.
- **Logs**: 작업과 연관된 로그 목록이 제공되는 탭입니다. Amazon CloudWatch Logs 액세스 권한이 있는 사용자는 로그 링크를 선택하여 CloudWatch에서 세부 로그 메시지를 확인할 수 있습니다. 로그가 생성되지 않는 작업 유형도 있습니다.
- **Information**: 작업 유형, 작업 이름, 작업 실행 시간 등 작업에 관한 기본적인 정보가 제공되는 탭입니다.

1. **ChurnModelProcess** 단계를 선택합니다. 이름이 **ChurnModelProcess**인 새 창이 표시됩니다.
1. **ChurnModelProcess** 창에서 이 파이프라인 단계와 연관된 탭을 검토합니다. 
    - **Input** 탭을 선택합니다. 이 탭에는 처리 단계에서 사용하는 파일 및 파라미터와 관련된 유용한 정보가 포함되어 있습니다. 파라미터 목록에는 작업에서 사용하는 인스턴스 유형과 이미지, 데이터 집합 위치, 코드 위치, 생성되는 여러 출력의 대상을 비롯한 세부 정보가 있습니다. 창 아래쪽으로 스크롤하여 작업으로 전달된 파일 입력을 확인합니다.
    - **Outputs** 탭을 선택합니다. 이 탭에는 파이프라인 단계에서 생성된 여러 파일과 해당 파일이 있는 위치가 표시됩니다. 이 파이프라인은 SageMaker Studio 기본 버킷에 모든 출력을 저장합니다.
    - **Logs** 탭을 선택합니다. 이 탭에는 작업에서 생성되는 로그가 표시됩니다. SageMaker Studio 내에서 로깅을 사용 가능하도록 설정하면 파이프라인 단계가 정상적으로 실행되지 않을 때 조사와 문제 해결을 빠르게 진행할 수 있습니다.
    - **Information** 탭을 선택합니다. 이 탭에서는 파이프라인 단계의 대략적인 개요가 제공됩니다. 개요에는 단계 유형, 단계 이름, 작업 로그 링크 등의 정보가 포함됩니다. 그리고 작업 실행 시간 및 작업을 실행하는 데 걸린 시간에 관한 세부 정보도 제공됩니다.
        - **Step Type**은 **Processing**입니다.

### 과제 2.3.2: 파이프라인 단계 세부 정보 검색

이어지는 단계에서는 DAG(Directed Acyclic Graph)에서 적절한 노드를 선택하여 지정된 파이프라인 단계에 관한 정보를 찾습니다. 아래에 나와 있는 문제의 해답을 찾는 과정에서 도움이 필요한 경우, 이 Python 노트북 끝부분에 포함되어 있는 정답이나 힌트를 참조할 수 있습니다.

1. **ChurnHyperParameterTuning** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?
    - 이 단계에서 생성된 **Overall Best Training Job**은 어느 것인가요?
1. **ChurnEvalBestModel** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?
    - 이전 단계에서 확인된 상위 모델을 평가하는 데 사용되는 Python 스크립트의 이름은 무엇인가요?
    - 이 파일은 어디에 있나요?
    - 이 단계의 결과는 어디에 작성되었나요?
1. **CheckAUCScoreChurnEvaluation** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?
    - **Evaluation outcome**은 무엇인가요?
1. **ChurnCreateModel** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?
    - 이 작업에서 로그가 생성되었나요?
1. **RegisterChurnModel** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?
    - AUC(곡선 아래 면적) 지표의 값은 무엇인가요?
1. **ChurnTransform** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?
    - 이 작업에서 로그가 생성되었나요?
    - 이 단계의 입력으로 사용된 파일은 무엇인가요?
1. **ChurnModelConfigFile** 단계에서 다음 세부 정보를 찾습니다. 
    - 이 작업을 실행하는 데 사용된 ProcessingInstanceType은 무엇인가요?
    - 이 단계의 **Step Type**은 무엇인가요?
1. **ClarifyProcessingStep** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 파일 출력은 무엇인가요?
    - 출력은 어디에 작성되었나요?

### 2.3.3: 파이프라인에서 모델 승인

1. 파이프라인 실행이 완료되면 **Model registry**에서 파이프라인이 생성한 모델을 확인합니다.
    SageMaker Studio에서 **SageMaker Home** 아이콘을 선택합니다.
    - **Models** 목록을 확장합니다.
    - **Model registry**를 선택합니다.
    - **ChurnModelPackageGroup** 모델 그룹을 엽니다.
    - **ChurnModelPackageGroup** 탭에서 **Versions** 테이블의 행을 마우스 오른쪽 버튼으로 클릭하여 열고 **Open model version**을 선택합니다. 모델 상태는 **Pending**입니다. **Execution** 값은 방금 완료된 파이프라인 실행의 이름입니다.

    각 탭에서 파이프라인에 관한 추가 세부 정보를 확인할 수 있습니다.
    - **Activity**: 모델의 활동이 표시되는 탭입니다. 모델을 수정한 후 경과한 시간과 이벤트 정보가 수록되어 있습니다.
    - **Model quality**: 모델 정확도 지표가 표시되는 탭입니다.
    - **Explainability**: 모델 특성의 중요도가 Shapley 값(SHAP) 단위로 표시되는 탭입니다.
    - **Bias report**: 모델에 포함될 수 있는 바이어스가 표시되는 탭입니다.
    - **Inference recommender**: 모델의 가성비를 높일 수 있는 권장 사항이 제공되는 탭입니다. 이 모델 패키지에서는 해당 기능이 지원되지 않으므로 이 탭에는 데이터가 포함되어 있지 않습니다.
    - **Load test**: 이 탭에서는 부하 테스트를 시작하여 다양한 인스턴스 유형을 실행해 보고 각 유형이 프로덕션 배포에 필요한 처리량 및 지연 시간 지표를 충족하는지 평가할 수 있습니다.
    - **Settings**: 모델이 생성된 시간, 모델이 생성된 파이프라인, 모델의 위치, 모델과 연관된 시험 구성 요소 등의 정보가 표시되는 탭입니다.

1. 모델을 승인합니다. 이 프로세스에서는 모델을 승인하기 전에 수동으로 검토해야 합니다. 하지만 파이프라인 내에서 모델 승인을 자동화할 수 있습니다.
    - <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status**</span>를 선택합니다.
    - 드롭다운 목록을 열고 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">**Approved**</span>를 선택합니다.
    - <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status**</span>를 선택합니다.
1. **ChurnModelPackageGroup** 탭을 닫습니다.

### 과제 2.3.4: AWS SDK를 사용하여 파이프라인 단계 확인

SageMaker Studio UI를 사용하여 파이프라인 세부 정보를 확인할 수 있을 뿐만 아니라 AWS SDK 명령을 사용할 수도 있습니다. 예를 들어, 다음 명령은 파이프라인 단계 목록을 반환합니다.

In [ ]:
#list-steps
RunPipeline.list_steps()

## 과제 2.4: 아티팩트 검토

다음 과제에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_10.ipynb** 탭을 옆쪽으로 끌거나 **lab_10.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 아티팩트를 살펴볼 때 지침을 표시할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 아티팩트 탐색을 완료한 후 **lab_10.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

### 과제 2.4.1: SageMaker Studio에서 아티팩트 검토

파이프라인이 실행되면 각 단계에서 파일, 훈련된 파라미터, 모델 등의 아티팩트가 생성됩니다. SageMaker Studio의 파이프라인에서 생성된 아티팩트를 확인할 수 있습니다.
1. **ChurnModelSMPipeline** 탭으로 돌아옵니다.
1. **Executions** 탭을 선택합니다.
1. 목록에 나열된 실행을 마우스 오른쪽 버튼으로 클릭하여 열고 **View trial components generated by execution**을 선택합니다. 

SageMaker Studio에서 이름이 **Trial Component List**인 새 탭이 열립니다. 

파이프라인에서 실행된 모든 작업의 목록이 이 탭에 표시됩니다.

각 시험 구성 요소에는 **Trial Component Type**이 지정되어 있습니다. 관련 시험 세부 정보의 여러 탭에서 제공되는 정보는 시험 구성 요소 유형에 따라 다릅니다. 시험 세부 정보에서 일부 구성 요소 유형의 데이터가 표시되지 않는 탭도 있습니다.

1. 작업 목록의 맨 위 행을 마우스 오른쪽 버튼으로 클릭하여 열고 **Open in trial details**를 선택합니다. 

SageMaker Studio에서 이름이 **Describe Trial Component**인 새 탭이 열립니다. 

**Trial Components** 아래에는 여러 탭이 있습니다. 파이프라인 단계에서 수행한 작업에 따라 일부 탭은 비어 있을 수도 있습니다. 

1. **Artifacts** 탭을 선택합니다. 단계에 사용된 입력과 출력의 세부 정보를 볼 수 있습니다.
1. **Explainability** 탭을 선택합니다. 이 탭에는 SageMaker Clarify에서 생성된 설명 가능성 보고서가 표시됩니다.
1. **Bias Report** 탭을 선택합니다. 이 탭에는 SageMaker Clarify에서 생성된 바이어스 보고서가 표시됩니다.

### 과제 2.4.2: 기본 S3 버킷에서 아티팩트 찾기
**참고**: 이 과제에는 AWS Management Console을 사용합니다. S3 버킷을 살펴본 후 SageMaker Studio가 열려 있는 브라우저 탭으로 돌아와 **lab_10.ipynb** 탭을 선택합니다.

1. 콘솔이 열려 있는 브라우저 탭에서 Amazon S3로 이동합니다.
1. **sagemaker-** 와 AWS 리전으로 시작하는 버킷 이름(예: **sagemaker-us-west-2-123456789**)을 선택합니다.
1. 이 버킷에 있는 폴더와 파일을 살펴봅니다. 이 버킷에는 데이터 집합, 처리 입력과 출력, SageMaker Clarify 결과 그리고 결과 모델에 사용된 기타 파일이 포함되어 있습니다.
1. SageMaker Studio가 열려 있는 브라우저 탭으로 돌아와 **lab_10.ipynb** 탭을 선택합니다.

## 과제 2.5(선택 사항): 파이프라인용 계보 빌드 및 검토

이전 과제에서는 SageMaker Clarify를 사용해 모델이 예측을 하는 방식을 설명하고 모델에 포함될 수 있는 바이어스를 파악하는 방법을 알아보았습니다. SageMaker Clarify를 사용하여 모델을 생성하는 데 사용되는 단계를 검색할 수도 있습니다. 모델을 감사할 때 이러한 방식으로 단계를 검색해야 하는 경우가 많습니다. 이 과제에서는 MLLineageHelper 모듈을 활용하여 현재 파이프라인 실행의 계보를 빌드합니다. ML Lineage Helper에 관한 자세한 내용은 [MLLineageHelper](https://github.com/aws-samples/ml-lineage-helper)를 참조하세요.

Amazon SageMaker ML Lineage Tracking은 ML 워크플로에 포함되어 있는 데이터 준비에서 모델 배포까지의 단계에 관한 정보를 생성하여 저장합니다. 이러한 추적 정보를 활용하면 워크플로 단계 재현, 모델 및 데이터 집합 계보 추적, 모델 거버넌스 및 감사 표준 설정 등의 작업을 수행할 수 있습니다.

### 과제 2.5.1: 세션 및 변수 설정

In [ ]:
#set-variables
fs_query = feature_group.athena_query()
fs_table = fs_query.table_name
query_string = 'SELECT * FROM "'+fs_table+'"'

### 과제 2.5.2: 모델 계보를 빌드하는 데 사용할 값 표시

구성에 포함되는 항목은 다음과 같습니다.
- **query_string**: MLLineageHelper 모듈로 전달되는 SageMaker Feature Store 쿼리입니다.
- **model_ref**: 평가 중인 모델의 이름입니다.
- **processing_job**: 모델을 생성한 처리 작업의 이름입니다.

In [ ]:
#print-values
print ('query_string:',query_string)

model_ref = sagemaker_client.list_models(SortBy = 'CreationTime', SortOrder = 'Descending')['Models'][0]['ModelName']
print ('model_ref:',model_ref)

processing_job = sagemaker_client.list_processing_jobs(SortBy = 'CreationTime', SortOrder = 'Descending', NameContains = 'ChurnModelProcess')['ProcessingJobSummaries'][0]['ProcessingJobName']
print ('processing_job:',processing_job)

processing_job_description = sagemaker_client.describe_processing_job(
    ProcessingJobName = processing_job
    )

### 과제 2.5.3: 처리 작업 설명

In [ ]:
#describe-processing-job
processing_job_description

### 과제 2.5.4: 모델을 생성하는 데 사용된 훈련 작업의 이름 표시

In [ ]:
#print-training-job
training_job_name  =  sagemaker_client.list_training_jobs(SortBy = 'CreationTime', SortOrder = 'Descending')['TrainingJobSummaries'][0]['TrainingJobName']
print (training_job_name)

### 과제 2.5.5: 모델의 계보 빌드

다음 오류가 표시되면 셀을 다시 실행합니다.
- **ClientError: An error occurred (ThrottlingException) when calling the UpdateArtifact operation (reached max retries: 4): Rate exceeded**(ClientError: UpdateArtifact 작업 호출 중 오류 발생(ThrottlingException)(최대 재시도 횟수 도달: 4): 빈도가 초과됨)

In [ ]:
#build-lineage
ml_lineage = MLLineageHelper()
lineage = ml_lineage.create_ml_lineage(training_job_name, model_name = model_ref,
                                       query = query_string, sagemaker_processing_job_description = processing_job_description,
                                       feature_group_names = [feature_group_name])

### 과제 2.5.6: 현재 시험 및 특성 그룹만 포함하도록 계보 제한

파이프라인은 여러 번 실행할 수 있습니다. 가장 최근의 훈련 작업 실행에서 세부 정보를 검색하려는 경우 현재 시험의 이름과 시험에서 사용하는 특성 그룹을 사용하여 계보 호출을 필터링합니다. 

이 셀을 실행하고 나면 모델을 생성하는 데 사용된 단계, 단계가 실행된 순서 그리고 파이프라인 내의 다른 작업을 실행하기 위해 실행된 작업이 하나의 테이블로 표시됩니다. **lineage_FS.csv** 파일에도 같은 정보가 작성됩니다. 이 파일을 다운로드하여 출력을 저장한 후 감사자 등 다른 팀원과 공유할 수 있습니다.

In [ ]:
#limit-lineage
trial_name = RunPipeline.describe()['PipelineExperimentConfig']['TrialName']
pat = str(trial_name)+'|'+'fg-FG'
df1 = lineage[lineage.apply(lambda x: any(x.str.contains(pat)),axis = 1)]
pd.set_option('display.max_colwidth', 120)
df1.to_csv('lineage_FS.csv') 
df1

### 과제 2.5.7: 모델 계보의 시각화 생성

In [ ]:
#visualize-lineage
plt.figure(3, figsize = (20, 14))
graph = nx.DiGraph()
graph.add_edges_from([(each[0], each[2]) for each in df1.values])
fig, ax = plt.subplots()
nx.draw_networkx(
    graph,
    node_size = 300,
    node_color = "orange",
    alpha = 0.65,
    font_size = 8,
    pos = nx.spring_layout(graph)
)
ax.set_facecolor('deepskyblue')
ax.axis('off')
fig.set_facecolor('deepskyblue')
plt.show()

## 과제 2.6: 파이프라인 제거

파이프라인을 삭제하려면 다음 셀을 실행합니다.

In [ ]:
#delete-pipeline
response = sagemaker_client.delete_pipeline(PipelineName = 'ChurnModelSMPipeline')
print (response)

### 마무리 

축하합니다! SageMaker 파이프라인을 사용하여 모델 생성과 등록을 자동화했습니다. 그리고 각 파이프라인 단계를 드릴다운하여 연관된 파라미터, 파일 및 로그를 확인하는 방법을 알아보았습니다. 또한 파이프라인에서 모델을 생성하는 데 사용한 자산을 확인하는 방법, 모델 레지스트리에서 모델을 찾는 방법 그리고 파이프라인이 생성할 수 있는 설명 가능성 및 바이어스 보고서를 찾아서 확인하는 방법도 살펴보았습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **Conclusion** 부분을 계속 진행합니다.

### 과제 2.3.1.1의 힌트와 정답
일반 힌트: **Step type**은 **Information** 탭에서 확인할 수 있습니다.

1. **ChurnHyperParameterTuning** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?</br>
    **정답**: Tuning</br>
    - 이 단계에서 생성된 **Overall Best Training Job**은 어느 것인가요? </br>
    **힌트**: 이 정보는 **Output** 탭에서 확인할 수 있습니다.</br>
    **정답**: 생성되는 모델 이름은 수강생별로 다릅니다. 예를 들어 056vhzs2vkxc-ChurnHy-TCAtUr16oV-001-17d5bd01과 비슷한 이름이 생성됩니다.
1. **ChurnEvalBestModel** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?
    **정답**: Processing</br>
    - 이전 단계에서 확인된 상위 모델을 평가하는 데 사용되는 Python 스크립트의 이름은 무엇인가요?</br>
    **힌트**: 이 정보는 **Input** 탭에서 확인할 수 있습니다.</br>
    **정답**: evaluate.py</br>
    - 이 파일은 어디에 있나요?</br>
    **힌트**: 이 정보는 **Input** 탭에서 확인할 수 있습니다.</br>
    **정답**: 해당 파일은 S3 버킷에 있습니다. 버킷의 경로는 예를 들어 s3://sagemaker-us-west-2-1234567890/input/code/evaluate.py와 비슷합니다.</br>
    - 이 단계의 결과는 어디에 작성되었나요?</br>
    **힌트**: 이 정보는 **Output** 탭에서 확인할 수 있습니다.</br>
    **정답**: 평가 결과는 S3 버킷에 작성되었습니다. 파일 경로는 예를 들어 s3://sagemaker-us-west-2-1234567890/output/evaluation과 비슷합니다.</br>
1. **CheckAUCScoreChurnEvaluation** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?</br>
    **정답**: Condition</br>
    - **Evaluation outcome**은 무엇인가요?</br>
    **힌트**: 이 정보는 **Output** 탭에서 확인할 수 있습니다.</br>
    **정답**: True
1. **ChurnCreateModel** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?</br>
    **정답**: Model</br>
    - 이 작업에서 로그가 생성되었나요?</br>
    **정답**: 아니요
1. **RegisterChurnModel** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?</br>
    **정답**: RegisterModel</br>
    - AUC 지표의 값은 무엇인가요?</br>
    **힌트**: 이 정보는 **Output** 탭에서 확인할 수 있습니다.</br>
    **정답**: 이 값은 수강생별로 다르지만 0.98에 가까운 값이 표시됩니다.
1. **ChurnTransform** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 **Step Type**은 무엇인가요?</br>
    **정답**: Transform</br>
    - 이 작업에서 로그가 생성되었나요?</br>
    **정답**: Yes</br>
    - 이 단계의 입력으로 사용된 파일은 무엇인가요?</br>
    **힌트**: 이 정보는 **Input** 탭에서 확인할 수 있습니다. 파일 이름을 확인하려면 창 아래쪽으로 스크롤해야 할 수 있습니다.</br>
    **정답**: model.tar.gz, sagemaker-xgboost:1.5-1-cpu-py3, batch.csv
1. **ChurnModelConfigFile** 단계에서 다음 세부 정보를 찾습니다. 
    - 이 작업을 실행하는 데 사용된 ProcessingInstanceType은 무엇인가요?</br>
    **힌트**: 이 정보는 **Input** 탭에서 확인할 수 있습니다.</br>
    **정답**: ml.m5.xlarge
    - 이 단계의 **Step Type**은 무엇인가요?</br>
    **정답**: Processing
1. **ClarifyProcessingStep** 단계에서 다음 세부 정보를 찾습니다.
    - 이 단계의 파일 출력은 무엇인가요?
    **힌트**: 이 정보는 **Output** 탭에서 확인할 수 있습니다.</br>
    **정답**: 출력은 바이어스 데이터입니다.
    - 출력은 어디에 작성되었나요?
    **정답**: 출력은 S3 버킷에 작성되었습니다. 경로는 예를 들어 s3://sagemaker-us-west-2-1234567890/clarify-output/bias와 비슷합니다.